# Week 4 — Part 04: Caching and Logging Lab

**Estimated time:** 90–120 minutes

---

## Pre-study (Self-learn)

Foundations Course assumes Self-learn is complete. If you need a refresher on observability and operations:

- [Foundations Course Pre-study index](../PRESTUDY.md)
- [Self-learn — Chapter 5: Resource Monitoring and Containerization](../self_learn/Chapters/5/Chapter5.md)

---

## What success looks like (end of Part 04)

- You can implement response caching to reduce costs.
- You can add structured logging for observability.
- You can trace requests through your system.

### Checkpoint

After running this notebook:
- You can cache LLM responses
- You can log requests and responses

## Learning Objectives

- Implement response caching
- Add structured logging
- Trace requests for debugging

### What this part covers
This notebook covers two practical tools for production LLM clients:

- **Caching** — avoid making the same expensive API call twice. A cache hit saves money and reduces latency.
- **Logging** — record what happened, when, and how long it took. Good logs turn debugging from guesswork into inspection.

**The critical rule for caching:** The cache key must include **every parameter that can change the output**. If you forget to include `temperature` in the key, you might return a cached deterministic response for a request that should be stochastic.

## Overview

Two practical realities of LLM APIs:

- calls can be expensive
- failures are hard to debug without logs

In this lab you will:

- build a safe cache key (include every field that can change output)
- demonstrate cache hit vs miss
- emit a minimal request log line (request_id, latency, success/failure)

If you want the deeper caching theory, use the Self-learn links at the top of the notebook.

### What this cell does
Defines `LLMRequest` — a frozen dataclass representing one LLM call — and `make_cache_key()` which converts it to a stable SHA-256 hash.

**Why a frozen dataclass?** Frozen means immutable — you can't accidentally modify a request after creating it. This is important for caching: the key must represent the request at the moment it was made, not a later mutated version.

**Why SHA-256 of sorted JSON?** `json.dumps(..., sort_keys=True)` produces deterministic output regardless of dict insertion order. SHA-256 compresses it to a fixed-length string suitable as a dict key or filename.

**What to notice:** Change any field in `LLMRequest` (even `temperature` from `0.0` to `0.1`) and the cache key changes completely. This is correct behavior — different temperature means different expected output.

## Caching

Cache when:

- the same request repeats
- you are iterating on downstream code

Cache key must include everything that changes output:

- model name
- system prompt
- user prompt
- temperature

Common cache pitfalls:

- forgetting system prompt / tool context in the key
- caching when temperature is high (outputs are intentionally stochastic)
- caching errors (you accidentally “remember” a failure)

### What this cell does
Implements `SimpleMemoryCache` — an in-memory dict-based cache — and demonstrates a cache miss followed by a cache hit.

**When to use memory cache vs file cache:**
- **Memory cache** — fast, zero I/O, but lost when the process exits. Good for within-session deduplication (e.g., same prompt called multiple times in one pipeline run).
- **File cache** (`SimpleFileCache` below) — persists across runs. Good for development iteration where you want to avoid re-calling the LLM every time you restart the notebook.

**Common pitfall:** Never cache errors. If an API call fails and you cache the failure, future calls will return the error immediately without retrying — even after the underlying issue is fixed.

In [ ]:
from __future__ import annotations

import hashlib
import json
import logging
import time
from dataclasses import asdict, dataclass
from pathlib import Path


@dataclass(frozen=True)
class LLMRequest:
    model: str
    system_prompt: str
    user_prompt: str
    temperature: float = 0.0


def make_cache_key(req: LLMRequest) -> str:
    # Key must include every field that can change the output.
    raw = json.dumps(asdict(req), sort_keys=True, ensure_ascii=False)
    return hashlib.sha256(raw.encode("utf-8")).hexdigest()


req = LLMRequest(model="demo", system_prompt="You are helpful.", user_prompt="Hello", temperature=0.0)
print("cache_key=", make_cache_key(req)[:12])

### What this cell does
Implements `SimpleFileCache` — a JSON-file-backed cache that persists across notebook restarts.

**How it works:** Reads the entire JSON file on every `get()` and `set()`. This is intentionally simple — not suitable for high-throughput production use, but perfect for development iteration where you want to avoid re-calling the LLM every time you restart.

**What to check:** After running, open `output/cache/llm_cache.json` — you should see the cached entry. Restart the kernel and run only the `get()` call — it should still return the cached value.

In [ ]:
from typing import Optional


class SimpleMemoryCache:
    def __init__(self) -> None:
        self._store = {}

    def get(self, key: str) -> Optional[str]:
        return self._store.get(key)

    def set(self, key: str, value: str) -> None:
        self._store[key] = value


cache = SimpleMemoryCache()
key = "k1"
print(cache.get(key))
cache.set(key, "value")
print(cache.get(key))

### What this cell does
Implements `logged_call()` — wraps an LLM call with structured logging that records `request_id`, `model`, `latency_s`, `prompt_len`, and `attempt`.

**Minimum viable log fields:**
- `request_id` — unique ID per call, so you can trace a specific request through logs
- `model` — which model was called (important when you have fallbacks)
- `latency_s` — how long the call took (spot slow calls immediately)
- `success/failure` — error rate tracking
- `error_type` — distinguish network errors from parsing errors from validation errors

**Why structured logging?** Plain `print()` statements are hard to filter and parse. Structured logs (key=value pairs or JSON) can be queried: "show me all calls where `latency_s > 10`" or "show me all `SCHEMA_ERROR` failures in the last hour."

In [ ]:
from typing import Dict, Optional


class SimpleFileCache:
    def __init__(self, path: Path) -> None:
        self.path = path
        self.path.parent.mkdir(parents=True, exist_ok=True)
        if not self.path.exists():
            self.path.write_text("{}", encoding="utf-8")

    def _read(self) -> Dict[str, str]:
        return json.loads(self.path.read_text(encoding="utf-8"))

    def _write(self, data: Dict[str, str]) -> None:
        self.path.write_text(json.dumps(data, ensure_ascii=False, sort_keys=True, indent=2), encoding="utf-8")

    def get(self, key: str) -> Optional[str]:
        data = self._read()
        return data.get(key)

    def set(self, key: str, value: str) -> None:
        data = self._read()
        data[key] = value
        self._write(data)


file_cache = SimpleFileCache(Path("output/cache/llm_cache.json"))
k = "demo"
print(file_cache.get(k))
file_cache.set(k, "hello")
print(file_cache.get(k))

## Logging (minimum viable request log)

A minimal request log should include:

- request id
- model
- latency
- success/failure
- failure location (network vs parsing vs validation)

Two extra fields that help later:

- prompt length (or token estimate)
- retry attempt count

In [ ]:
logger = logging.getLogger("demo")
logging.basicConfig(level=logging.INFO)


def fake_llm_call(text: str) -> str:
    time.sleep(0.05)
    return text.upper()


def logged_call(request_id: str, req: LLMRequest) -> str:
    t0 = time.time()
    try:
        out = fake_llm_call(req.user_prompt)
        logger.info(
            "llm_call_ok",
            extra={
                "request_id": request_id,
                "model": req.model,
                "latency_s": time.time() - t0,
                "prompt_len": len(req.system_prompt) + len(req.user_prompt),
                "attempt": 0,
            },
        )
        return out
    except Exception as e:
        logger.warning(
            "llm_call_failed",
            extra={
                "request_id": request_id,
                "model": req.model,
                "latency_s": time.time() - t0,
                "attempt": 0,
                "error_type": type(e).__name__,
            },
        )
        raise


print(logged_call("req_001", req))

In [ ]:
def cached_call(cache_obj: SimpleMemoryCache, req: LLMRequest) -> str:
    key = make_cache_key(req)
    hit = cache_obj.get(key)
    if hit is not None:
        logger.info("llm_cache_hit", extra={"model": req.model})
        return hit

    out = fake_llm_call(req.user_prompt)
    cache_obj.set(key, out)
    logger.info("llm_cache_set", extra={"model": req.model})
    return out


cache2 = SimpleMemoryCache()
print(cached_call(cache2, req))
print(cached_call(cache2, req))

In [ ]:
def make_cache_key_todo(req: LLMRequest) -> str:
    # TODO: extend the key so it would remain correct if you add fields like:
    # - top_p
    # - max_tokens
    # - tool schema / tool definitions
    # - few-shot examples
    return make_cache_key(req)


def should_cache(req: LLMRequest) -> bool:
    # TODO: implement policy, e.g.
    # - cache only if temperature == 0.0
    # - avoid caching very large prompts
    return req.temperature == 0.0 and (len(req.system_prompt) + len(req.user_prompt)) <= 10_000


print("make_cache_key_todo:", make_cache_key_todo(req)[:12])
print("should_cache:", should_cache(req))

## References

- `functools.lru_cache`: https://docs.python.org/3/library/functools.html#functools.lru_cache
- Python logging: https://docs.python.org/3/library/logging.html

## Appendix: Solutions (peek only after trying)

Reference implementations for `make_cache_key_todo` and `should_cache`.

In [ ]:
def make_cache_key_todo(req: LLMRequest) -> str:
    # Safe approach: serialize all known fields deterministically.
    raw = json.dumps(asdict(req), sort_keys=True, ensure_ascii=False)
    return hashlib.sha256(raw.encode("utf-8")).hexdigest()


def should_cache(req: LLMRequest) -> bool:
    # Conservative policy for reproducibility + correctness.
    if req.temperature != 0.0:
        return False
    prompt_len = len(req.system_prompt) + len(req.user_prompt)
    return prompt_len <= 10_000


print("solution key:", make_cache_key_todo(req)[:12])
print("solution should_cache:", should_cache(req))